## Gateways reached by a Sensor

In [1]:
from datetime import datetime, timedelta
import subprocess
import pandas as pd
from dateutil.parser import parse
from dateutil import tz
from label_map import dev_id_lbls, gtw_lbls

In [4]:
DEVICES = [
    '3148 E 19th, Anc',
    '3414 E 16th, Anc',
    '3424 E 18th, Anc',
    '122 N Bliss',
    '122 N Bliss Unit',
    '1826 Columbine, Anc',
    'Phil ELT-2 3692',
    'Phil LT22222 436E',
    'Phil CO2 26D8',
]

# Days of Data to Show
DAYS = 4

GATEWAY_FILE = '~/gateways.tsv'

In [3]:
# Make DateTime objects for time period analyze
tz_ak = tz.gettz('US/Alaska')
start_ts = (datetime.now(tz_ak) - timedelta(days=DAYS)).replace(
    tzinfo=None, minute=0, second=0, microsecond=0)
end_ts = datetime.now(tz_ak).replace(
    tzinfo=None, minute=0, second=0, microsecond=0)
start_ts, end_ts

(datetime.datetime(2021, 10, 16, 7, 0), datetime.datetime(2021, 10, 20, 7, 0))

In [5]:
# Pick a device to use for testing the script
device = DEVICES[3]

df = pd.read_csv(GATEWAY_FILE, 
    sep='\t', 
    parse_dates=['ts', 'ts_hour'],
    index_col='ts',
    low_memory=False)
df = df.loc[str(start_ts):]
df['dev_id'] = df.dev_id.map(dev_id_lbls)
df.query('dev_id == @device', inplace=True)

def gtw_map(gtw_eui):
    return gtw_lbls.get(gtw_eui, gtw_eui)

df['gateway'] = df.gateway.map(gtw_map)
df.head()

<ipython-input-5-b6feac3e82ec>:9: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df = df.loc[str(start_ts):]


,dev_id,ts_day,ts_hour,counter,gateway,snr,rssi,data_rate
ts,,,,,,,,
2021-10-16 07:08:06,122 N Bliss,2021-10-16,2021-10-16 07:00:00,40001,dlos8-airport-heights-e2b4,8.2,-52,SF10BW125
2021-10-16 07:18:06,122 N Bliss,2021-10-16,2021-10-16 07:00:00,40002,dlos8-airport-heights-e2b4,12.5,-46,SF10BW125
2021-10-16 07:18:06,122 N Bliss,2021-10-16,2021-10-16 07:00:00,40002,Kasilof,-11.0,-95,SF10BW125
2021-10-16 07:28:06,122 N Bliss,2021-10-16,2021-10-16 07:00:00,40003,dlos8-airport-heights-e2b4,12.8,-47,SF10BW125
2021-10-16 07:38:06,122 N Bliss,2021-10-16,2021-10-16 07:00:00,40004,dlos8-airport-heights-e2b4,12.0,-44,SF10BW125


In [6]:
# Determine the "Any" gateway counts by dropping duplicate readings
df_any = df[['ts_hour', 'counter']].drop_duplicates(subset=['counter'])
df_any_count = df_any.groupby('ts_hour').count()
df_any_count.columns = ['Any' ]
df_any_count

,Any
ts_hour,
2021-10-16 07:00:00,6
2021-10-16 08:00:00,6
2021-10-16 09:00:00,6
2021-10-16 10:00:00,6
2021-10-16 11:00:00,6
...,...
2021-10-20 03:00:00,6
2021-10-20 04:00:00,6
2021-10-20 05:00:00,6


In [7]:
# Determine counts for individual gateways
df_cts = pd.pivot_table(df, index='ts_hour', columns='gateway', values='counter', aggfunc='count')
df_cts

gateway,Kasilof,dlos8-airport-heights-e2b4
ts_hour,,
2021-10-16 07:00:00,2.0,6.0
2021-10-16 08:00:00,2.0,6.0
2021-10-16 09:00:00,NaN,6.0
2021-10-16 10:00:00,NaN,6.0
2021-10-16 11:00:00,NaN,6.0
...,...,...
2021-10-20 03:00:00,NaN,6.0
2021-10-20 04:00:00,NaN,6.0
2021-10-20 05:00:00,NaN,6.0


In [8]:
# Combine the two DataFrames, horizontally (combine columns)
df_final = pd.concat([df_any_count, df_cts], axis=1)

# Make a new index that fills in any missing hours
new_ix = pd.date_range(start_ts, end_ts, freq='1H')
df_final = df_final.reindex(new_ix)

# Replace NaN values with zero and then convert values to integers
df_final.fillna(0, inplace=True)
df_final = df_final.astype('int32')

df_final = df_final[:-1]    # drop last hour because likely incomplete

# Convert index into a string so we can change drop the seconds from display
df_final.index = df_final.index.strftime("%Y-%m-%d %H:%M")

df_final

,Any,Kasilof,dlos8-airport-heights-e2b4
2021-10-16 07:00,6,2,6
2021-10-16 08:00,6,2,6
2021-10-16 09:00,6,0,6
2021-10-16 10:00,6,0,6
2021-10-16 11:00,6,0,6
...,...,...,...
2021-10-20 02:00,6,0,6
2021-10-20 03:00,6,0,6
2021-10-20 04:00,6,0,6
2021-10-20 05:00,6,0,6


In [9]:
def color_cells(val):
    color_scale = {
        0: '#FF3131',
        1: '#FFFF00',
        2: '#FFD822',
    }    
    color = color_scale.get(val, '#EEEEEE')
    return 'background: %s' % color

s = df_final.style
s.applymap(color_cells)
s.set_properties(**{'width': '70px', 'text-align': 'center'})
styles = [
    dict(selector="td", props=[('padding', "0px")]),
]         
s.set_table_styles(styles)

s

,Any,Kasilof,dlos8-airport-heights-e2b4
2021-10-16 07:00,6,2,6
2021-10-16 08:00,6,2,6
2021-10-16 09:00,6,0,6
2021-10-16 10:00,6,0,6
2021-10-16 11:00,6,0,6
2021-10-16 12:00,6,1,6
2021-10-16 13:00,3,0,3
2021-10-16 14:00,3,1,3
2021-10-16 15:00,4,0,4
2021-10-16 16:00,2,0,2
